In [1]:
import torch

In [3]:
from torchvision.models import mobilenet_v2
import torch.nn.functional as F # for bce loss function
from torch.utils.data import DataLoader
import torch.nn as nn

#import matplotlib.pyplot as plt
import numpy as np
from typing import Callable, Tuple, List
from tqdm import tqdm

     
BATCH_SIZE = 32
INPUT_CHANNELS = 12
RES_BLOCK_INPUT_CHANNELS = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [30]:
# replacing: 
# tf.keras.applications.MobileNetV2(input_shape=[32, 32, 12], include_top=False, weights=None)
class Downstack(nn.Module):
    def __init__(self, input_channels, kernel_size, stride, padding):
        super().__init__() # initialize the class as a pytorch module

        self.base_model = mobilenet_v2(weights=None) # torch imp only takes weights as parameter
        # torch auto runs the classifier in the forward pass so have to replace those layers with identity to
        # get equivalent to include_top=False
        self.base_model.classifier = nn.Identity() 

        # changing default input size from 3 (RGB) to 12 for wildfire images
        self.base_model.features[0][0] = nn.Conv2d(
            in_channels = input_channels,
            out_channels = RES_BLOCK_INPUT_CHANNELS,
            kernel_size = kernel_size,
            stride = stride,
            padding = padding,
            bias = False
        )

        layrs = torch.randn(2, 12, 32, 32)
        for i, layer in enumerate(self.base_model.features):
            layrs = layer(layrs)
            print(i, layrs.shape)


        self.base_model.eval()
        print(self.base_model)
        print("xxxxxxxxxxxxxxxxxxxxxxx")
        print(self.base_model.features[2].conv[0][0])
    
    def forward(self, x):
        # return a dictionary of the output layers
        pass


def main():
    base = Downstack(12, 3, 2, 1)
    

main()

0 torch.Size([2, 32, 16, 16])
1 torch.Size([2, 16, 16, 16])
2 torch.Size([2, 24, 8, 8])
3 torch.Size([2, 24, 8, 8])
4 torch.Size([2, 32, 4, 4])
5 torch.Size([2, 32, 4, 4])
6 torch.Size([2, 32, 4, 4])
7 torch.Size([2, 64, 2, 2])
8 torch.Size([2, 64, 2, 2])
9 torch.Size([2, 64, 2, 2])
10 torch.Size([2, 64, 2, 2])
11 torch.Size([2, 96, 2, 2])
12 torch.Size([2, 96, 2, 2])
13 torch.Size([2, 96, 2, 2])
14 torch.Size([2, 160, 1, 1])
15 torch.Size([2, 160, 1, 1])
16 torch.Size([2, 160, 1, 1])
17 torch.Size([2, 320, 1, 1])
18 torch.Size([2, 1280, 1, 1])
MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(12, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(

In [ ]:
# to create upstack: replacing pix2pix.upsample layers
# pix2pix layers are just a conv transpose layer, a batchnorm layer, and a relu layer 
# (dropout optional but not used in NDWS model)
def upsample(input, output, kernel_size=3, stride=2, padding=1):
    block = nn.Sequential(
        nn.ConvTranspose2d(in_channels=input, out_channels=output, kernel_size=kernel_size, 
                           stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(output),
        nn.ReLU()
    )
    nn.init.normal_(block[0].weight, mean=0.0, std=0.02) #initializer?
    return block    

In [ ]:
class convAutoencoder(nn.Module):
    def __init__(self, input_channels):
        super.__init__() # initialize as pytorch module

        downstack_model = Downstack(input_channels, kernel_size=3, stride=2, padding=1)

        upstack = nn.Sequential(
                    upsample(1280, 512),
                    upsample(512, 256),
                    upsample(256, 128),
                    upsample(128, 64)
        )

        # up_dimensions = [# in  out
        #                 [1280, 512], 
        #                 [512, 256], 
        #                 [256, 128], 
        #                 [128, 64],
        #                 ]
        # upstack_blocks = []
        # for input, output in up_dimensions:
        #     upstack_blocks.append(upsample(input, output))
        # upstack = nn.Sequential(*upstack_blocks)

    def forward(self, x):
        pass

                

## data prep
## data visualization methods
do we need these??? ^

## evaluation metrics (DONE)
- IoU: Intersection over Union. measures overlap between prediction region and actual (ground truth) region. Scores from 0 to 1, where 1 is perfect overlap and 0 is no overlap. Usually used to determine whether bounding box is correct
- Recall
- Precision

In [7]:
"""
IoU metric
Calculation of intersection over union metric.
    
Args:
    real_mask (Tensor): Ground-truth mask
    predicted_mask (Tensor): Mask predicted by model
Returns:
    (float): IoU metric value

CHANGES: 
    - changed "tf" to "torch" in method header and real_mask line
    - added comments
"""
def IoU_metric(real_mask: torch.Tensor, predicted_mask: torch.Tensor) -> float: 
    # replacing neg values: torch.where(condition, choose-True, choose-False)
    # when the value is pos (>=0), keep the value from real_mastorch. otherwise, replace with 0
    real_mask = torch.where(real_mask>=0, real_mask, 0)

    # calculates the intersection and union between real and predicted by using a log AND and OR functions from numpy
    intersection = np.logical_and(real_mask, predicted_mask)
    union = np.logical_or(real_mask, predicted_mask)

    # if there is no object in either mask (both are entirely 0s), return 1 since IoU for 
    # empty masks would be perfect
    if np.sum(union) == 0:
        return 1
    
    # else, calculate and return intersection over union (IoU)
    return np.sum(intersection) / np.sum(union)


"""
Calculation of recall metric.
    
Args:
    real_mask (Tensor): Ground-truth mask
    predicted_mask (Tensor): Mask predicted by model
Returns:
    (float): recall metric value

CHANGES:
    - changed tf to torch
"""
def recall_metric(real_mask: torch.Tensor, predicted_mask: torch.Tensor) -> float:

    real_mask = torch.where(real_mask < 0, 0, real_mask)
    
    true_positives = np.sum(np.logical_and(real_mask, predicted_mask))
    actual_positives = np.sum(real_mask)
    if actual_positives == 0:
        return 1
    
    return true_positives / actual_positives

"""
Calculation of precision metric.
    
Args:
    real_mask (Tensor): Ground-truth mask
    predicted_mask (Tensor): Mask predicted by model
Returns:
    (float): precision metric value

CHANGES:
    - changed tf to torch
"""
def precision_metric(real_mask: torch.Tensor, predicted_mask: torch.Tensor) -> float:
    real_mask = torch.where(real_mask < 0, 0, real_mask)
    
    true_positives = np.sum(np.logical_and(real_mask, predicted_mask))
    predicted_positives = np.sum(predicted_mask)
    if predicted_positives == 0:
        return 1
    
    return true_positives / predicted_positives

## loss functions (DONE)
- dice coefficient: metric used to evaluate similarity between sets, particularly in image segmentation. Dice coeff is calculated to be 2 times the intersection of the ground truth and predicted, over ground truth plus predicted. 
$$
\text{Dice Coefficient} = \frac{2 \times |A \cap B|}{|A| + |B|}
$$

- weighted binary cross entropy

- BCE Dice loss 
regular dice loss:  
$$ \text{Dice Loss} = 1 - \text{Dice Coefficient}$$  
BCE dice loss adds dice loss and wBCE loss

In [8]:
"""
Dice loss function calculator.
    
Args:
    y_true (Tensor): 
    y_pred (Tensor):
Returns:
    (Tensor): Dice loss for each element of a batch.

CHANGES:
    - changed tf to torch in method header
    - changed K to torch throughout
"""
def dice_coef(y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
    smooth = 1e-6
    y_true_f = torch.reshape(y_true, (BATCH_SIZE, -1))
    y_pred_f = torch.reshape(y_pred, (BATCH_SIZE, -1))
    intersection = torch.sum(y_true_f * y_pred_f, axis=1)
    return 1 - (2. * intersection + smooth) / (torch.sum(y_true_f, axis=1) + torch.sum(y_pred_f, axis=1) + smooth)


"""
Calculates weighted binary cross entropy. The weights are fixed.
    
This can be useful for unbalanced catagories.

Adjust the weights here depending on what is required.

For example if there are 10x as many positive classes as negative classes,
    if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
    will be penalize 10 times as much as false negatives.

Args:
    true (Tensor): Ground-truth values
    pred (Tensor): Predited values
    weight_zero (float): Weight of class 0 (no-fire)
    weight_one (float): Weight of class 1 (fire)

Returns: 
    (float) : value for weighted binary cross entropy
CHANGES:
    - changed tf to torch in method header
    - changed K to torch throughout
    - changed keras BCE method to torch.nn.functional.binary_cross_entropy
    
"""
def weighted_bincrossentropy(true: torch.Tensor, pred: torch.Tensor, weight_zero: float = 0.01, weight_one: float = 1) -> float:
  
    # calculate the binary cross entropy
    # using torch.nn.functional.binary_cross_entropy, set reduction='none' to keep individual losses in a tensor
    # rather than taking mean 
    bin_crossentropy = F.binary_cross_entropy(true, pred, reduction='none')
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy 
    
    return torch.mean(weighted_bin_crossentropy, axis=1)


"""
BCE loss function calculator.

Args:
    y_true (Tensor): 
    y_pred (Tensor):
Returns:
    (Tensor): Mean BCE Dice loss over a batch.

CHANGES:
    - changed tf to torch
    - originally returned keras.reduce_weighted_loss(loss), but without additional args all that did was 
      perform a sum operation. Replaced it with torch.sum since there is no torch equivalent to reduce_weighted_loss
"""
def bce_dice_loss(y_true: torch.Tensor, y_pred: torch.Tensor):    
    y_true_f = torch.reshape(y_true, (BATCH_SIZE, -1))
    y_pred_f = torch.reshape(y_pred, (BATCH_SIZE, -1))
    return torch.sum(weighted_bincrossentropy(y_true_f, y_pred_f) + dice_coef(y_true, y_pred))

# Evaluation loop: done?

In [9]:
"""
Loads dataset according to file pattern and evaluates model's predictions on it.

Parameters:
    model (Callable[[tf.Tensor], tf.Tensor]): Function for model inference.
    eval_dataset (tf.dataDataset): Dataset for evaluation.

Returns:
    Tuple[float, float, float, float]: IoU score, recall score, precision score and mean loss.

CHANGES:
    - changed tf to torch
    - in method header, imported DataLoader from torch.utils and changed  eval_dataset: tf.data.Dataset) to DataLoader
    - changed tf.expand_dims(tf.cast(predictions, tf.float32), axis=-1) to predictions.float().unsqueeze(-1)
    in losses.append
"""
def evaluate_model(prediction_function: Callable[[torch.Tensor], torch.Tensor],
                   eval_dataset: DataLoader) -> Tuple[float, float, float, float]:
    IoU_measures = []
    recall_measures = []
    precision_measures = []
    losses = []
    
    for inputs, labels in tqdm(eval_dataset):
        # Prediction shape (N, W, H)
        predictions = prediction_function(inputs)
        for i in range(inputs.shape[0]):
            IoU_measures.append(IoU_metric(labels[i, :, :,  0], predictions[i, :, :]))
            recall_measures.append(recall_metric(labels[i, :, :,  0], predictions[i, :, :]))
            precision_measures.append(precision_metric(labels[i, :, :,  0], predictions[i, :, :]))
        labels_cleared = torch.where(labels < 0, 0, labels)
        losses.append(bce_dice_loss(labels_cleared, predictions.float().unsqueeze(-1)
))
            
    mean_IoU = np.mean(IoU_measures)
    mean_recall = np.mean(recall_measures)
    mean_precision = np.mean(precision_measures)
    mean_loss = np.mean(losses)
    return mean_IoU, mean_recall, mean_precision, mean_loss

# Torch Model

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torchvision.ops as ops
from models import mobilenet_v2


base_model = mobilenet_v2(weights=None)

# setting classifier to Identity() removes the FC layers at end of classifier and replaces them with a "placeholder" operator
# which doesn't manipulate the outputs at all. Equivalent to keras include_top=False
base_model.classifier = nn.Identity() 

# from torchvision source code, default first layer for mobilenetv2 expects 3 input channels. have to modify this first layer to 
# take 12 input channels instead. https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py
# [0][0] modifies first layer in the first block of the feature extractor layer array. 
# From source code, out_channels=32, norm_layer=batchnormd, activationlayer=relu6
base_model.features[0][0] = ops.Conv2dNormActivation(12, 32, stride=2, padding=1, norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU6)

#print(base_model)
layer = base_model.get_submodule("features.2.conv.0.2")
print(layer)
# for name, module in base_model.features.named_modules():
#     if isinstance(module, nn.ReLU6):
#         print(f"Found ReLU6 layer at {name}")

# for name, module in base_model.named_modules():
#     print(name, module)


class ModMobileNetV2(nn.Module):
    def __init__(self, input_channels=12, include_top=False):
        super().__init__()
        self.model = mobilenet_v2(weights=None)
        # 




ReLU6(inplace=True)


In [ ]:
import torchvision.models as models
import torch.nn as nn
import torchvision.ops as ops

def cnn_ae_model():
    base_model = models.mobilenet_v2(weights=None, pretrained=False)

    # setting classifier to Identity() removes the FC layers at end of classifier and replaces them with a "placeholder" operator
    # which doesn't manipulate the outputs at all. Equivalent to keras include_top=False
    base_model.classifier = nn.Identity() 

    # from torchvision source code, default first layer for mobilenetv2 expects 3 input channels. have to modify this first layer to 
    # take 12 input channels instead. https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py
    # [0][0] modifies first layer in the first block of the feature extractor layer array. 
    # From source code, out_channels=32, norm_layer=batchnormd, activationlayer=relu6
    base_model.features[0][0] = ops.Conv2dNormActivation(12, 32, stride=2, padding=1, norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU6)

  




# Torch Training loop
### 

not finished converting! changes made so far:  
- tf.data.Dataset to DataLoader in method header
- tf.keras.optimizers.Adam() to torch.optim.Adam() for optimizer
- tf to torch for .where()

In [ ]:
"""
Trains a model using train dataset. (Save weights of model with best IoU)

Args:
    model (Model): Model to train.
    train_dataset (Dataset): Training dataset.
    epochs (int): Number of epochs
Returns:
    Tuple[List[float], List[float]]: Train losses and Validation losses

CHANGES:
    - tf.data.Dataset to DataLoader in method header
    - tf.keras.optimizers.Adam() to torch.optim.Adam() for optimizer
    - tf to torch for .where()
    - removed GradientTape() call, replace all gradient stuff with "optimizer.zero_grad(), loss.backward(), optimizer.step()"
"""

def train_model(model: Model, train_dataset: DataLoader, epochs:int=10) -> Tuple[List[float], List[float]]:
    loss_fn = bce_dice_loss
    optimizer = torch.optim.Adam()
    batch_losses = []
    val_losses = []
    best_IoU = 0.0
    
    for epoch in range(epochs):
        losses = []
        print(f'Epoch {epoch+1}/{epochs}')
        # Iterate through the dataset
        progress = tqdm(train_dataset)
        for images, masks in progress:
            #with tf.GradientTape() as tape:

            # Forward pass
            predictions = model(images, training=True)
            label = torch.where(masks < 0, 0, masks)
            # Compute the loss
            loss = loss_fn(label, predictions)
            losses.append(loss.numpy())
            progress.set_postfix({'batch_loss': loss.numpy()})

            # Compute gradients
            optimizer.zero_grad()
            loss.backware()
            optimizer.step()
            #gradients = tape.gradient(loss, model.trainable_variables)
            # Update the model's weights
            #optimizer.apply_gradients(zip(gradients, model.trainable_variables)) <-- what here?

        # Evaluate model
        print("Evaluation...")
        IoU, recall, precision, val_loss = evaluate_model(lambda x: torch.where(model.predict(x) > 0.5, 1, 0)[:,:,:,0], validation_dataset)
        print("Validation set metrics:")
        print(f"Mean IoU: {IoU}\nMean precision: {precision}\nMean recall: {recall}\nValidation loss: {val_loss}\n")
        # Save best model
        if IoU > best_IoU:
            best_IoU = IoU
            model.save_weights("best.h5")
        
        # Print the loss for monitoring
        print(f'Epoch: {epoch}, Train loss: {np.mean(losses)}')
        batch_losses.append(np.mean(losses))
        val_losses.append(val_loss)
    
    print(f"Best model IoU: {best_IoU}")
    return batch_losses, val_losses

# Set reproducability
tf.random.set_seed(1337)

segmentation_model = build_CNN_AE_model()
train_losses, val_losses = train_model(segmentation_model, train_dataset, epochs=15)

# Plot Loss functions

# Testing

# Metrics on test set